# 2 字符串和文本

Contents:

* 2.1 使用多个界定符分割字符串
* 2.2 字符串开头或结尾匹配
* 2.3 用Shell通配符匹配字符串
* 2.4 字符串匹配和搜索
* 2.5 字符串搜索和替换
* 2.6 字符串忽略大小写的搜索替换
* 2.7 最短匹配模式
* 2.8 多行匹配模式
* 2.9 将Unicode文本标准化
* 2.10 在正则式中使用Unicode
* 2.11 删除字符串中不需要的字符
* 2.12 审查清理文本字符串
* 2.13 字符串对齐
* 2.14 合并拼接字符串
* 2.15 字符串中插入变量
* 2.16 以指定列宽格式化字符串
* 2.17 在字符串中处理html和xml
* 2.18 字符串令牌解析
* 2.19 实现一个简单的递归下降分析器
* 2.20 字节字符串上的字符串操作

## 2.1 使用多个界定符分割字符串

problem: 你需要将一个字符串分割为多个字段，但是分隔符(还有周围的空格) 并不是固定的。

ans: string 对象的split() 方法, re.split()

In [1]:
line = 'asdf fjdk; afed, fjek,asdf, foo'
import re
re.split(r'[;,\s]\s*', line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

In [10]:
# 括号捕获分组
fields = re.split(r'(;|,|\s)\s*', line)
print(fields)

# 保留分割字符串, 构造新字符串
values = fields[::2]    # start=0, step=2, 偶数号
delimiters = fields[1::2] + ['']    # start=1, step=2, 奇数号
print(values)
print(delimiters)

# Reform the line using the same delimiters
''.join(v+d for v,d in zip(values, delimiters))

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']
['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']
[' ', ';', ',', ',', ',', '']


'asdf fjdk;afed,fjek,asdf,foo'

## 2.2 字符串开头或结尾匹配

problem: 你需要通过指定的文本模式去检查字符串的开头或者结尾，比如文件名后缀，URL Scheme 等等。

ans: str.startswith() 或者是str.endswith() 方法

In [15]:
# 检查字符串开头或结尾
filename = 'spam.txt'
filename.endswith('.txt')
filename.startswith('file:')
url = 'https://www.python.org'
url.startswith('https:')

True

In [21]:
# 检查多种匹配可能, use tuple
import os

filenames = os.listdir('.')
print(filenames)
ipynb = [name for name in filenames if name.endswith('.ipynb')]
print(ipynb)
any(name.endswith('.ipynb') for name in filenames)

['2.string and text.ipynb']
['2.string and text.ipynb']


True

In [25]:
from urllib.request import urlopen

def read_data(name):
    if name.startswith(('http:', 'https:', 'ftp:')):  # input tuple
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()

# # input tuple
choices = ['https:', 'ftp:']
url = 'https://www.python.org'
url.startswith(tuple(choices)) 
read_data(url)

True

In [29]:
# slice
filename = 'spam.txt'
filename[-4:] == '.txt'
url = 'https://www.python.org'
url[:6] == 'https:' or url[:5] == 'http:' or url[:4] == 'ftp:'

# re
import re
re.match('http:|https:|ftp:', url)

<re.Match object; span=(0, 6), match='https:'>

## 2.3 用Shell 通配符匹配字符串

problem: 你想使用Unix Shell 中常用的通配符(比如\*.py , Dat\[0-9\]\*.csv 等) 去匹配文本字符串。

ans: fnmatch 模块提供了两个函数——fnmatch() 和fnmatchcase() ，可以用来实现这样的匹配。

In [32]:
from fnmatch import fnmatch, fnmatchcase

fnmatch('foo.txt', '*.txt')
fnmatch('foo.txt', '?txt')
fnmatch('Dat45.csv', 'Dat[0-9]*')
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

In [34]:
# fnmatch() 函数使用底层操作系统的大小写敏感规则(不同的系统是不一样的) 来匹配模式
fnmatch('foo.txt', '*.TXT')
# fnmatchcase
fnmatchcase('foo.txt', '*.TXT')

True

In [36]:
# 处理非文件名的字符串
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY',
]

from fnmatch import fnmatchcase
ls = [addr for addr in addresses if fnmatchcase(addr, '*ST')]
print(ls)
ls = [addr for addr in addresses if fnmatchcase(addr, '54[0-9][0-9] *CLARK*')]
print(ls)

['5412 N CLARK ST', '1060 W ADDISON ST', '2122 N CLARK ST']
['5412 N CLARK ST']


## 2.4 字符串匹配和搜索

problem: 你想匹配或者搜索特定模式的文本。

ans: str.find(), str.endswith(), str.startswith(), etc. | re

In [2]:
text = 'yeah, but no, but yeah, but no, but yeah'
# Exact match
print(text == 'yeah')
# Match at start or end
print(text.startswith('yeah'))
print(text.endswith('no'))
# Search for the location of the first occurrence
print(text.find('no'))

False
True
False
10


In [2]:
# 对于复杂的匹配需要使用正则表达式和 re 模块
text1 = '11/27/2012'
text2 = 'Nov 27, 2012'

import re
# Simple matching: \d+ means match one or more digits
if re.match(r'\d+/\d+/\d+', text1):
    print('yes')
else: 
    print('no')

if re.match(r'\d+/\d+/\d+', text2):
    print('yes')
else:
    print('no')

yes
no


In [5]:
# 如果你想使用同一个模式去做多次匹配，你应该先将模式字符串预编译为模式对象
datepet = re.compile(r'\d+/\d+/\d+')
if datepet.match(text1):
    print('yes')
else: 
    print('no')

if datepet.match(text2):
    print('yes')
else:
    print('no')

yes
no


In [6]:
# match() 总是从字符串开始去匹配，
# 如果你想查找字符串任意部分的模式出现位置， 使用 findall() 方法去代替。
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
datepet.findall(text)

['11/27/2012', '3/13/2013']

In [15]:
# 在定义正则式的时候，通常会利用括号去捕获分组
datepet = re.compile(r'(\d+)/(\d+)/(\d+)')

# 捕获分组可以使得后面的处理更加简单，因为可以分别将每个组的内容提取出来
m = datepet.match('11/27/2012')
print(m)

# Extract the contents of each group
print(m.group(0))
print(m.group(1))
print(m.group(2))
print(m.groups())

month, day, year = m.groups()
# Find all matches (notice splitting into tuples)
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
datepet.findall(text)
for month, day, year in datepet.findall(text):
    print('{}-{}-{}'.format(year, month, day))

<re.Match object; span=(0, 10), match='11/27/2012'>
11/27/2012
11
27
('11', '27', '2012')
2012-11-27
2013-3-13


In [16]:
# findall() 方法会搜索文本并以列表形式返回所有的匹配
# 如果你想以迭代方式返回匹配，可以使用 finditer() 方法来代替
for m in datepet.finditer(text):
    print(m.groups())

('11', '27', '2012')
('3', '13', '2013')


In [18]:
# 需要注意的是 match() 方法仅仅检查字符串的开始部分
m = datepet.match('11/27/2012abcdef')
print(m)
print(m.group())

# 如果你想精确匹配，确保你的正则表达式以$结尾
datepet = re.compile(r'(\d+)/(\d+)/(\d+)$')
datepet.match('11/27/2012abcdef')
datepet.match('11/27/2012')

<re.Match object; span=(0, 10), match='11/27/2012'>
11/27/2012


<re.Match object; span=(0, 10), match='11/27/2012'>

In [19]:
# 如果你仅仅是做一次简单的文本匹配/搜索操作的话，可以略过编译部分，直接使用 re 模块级别的函数
re.findall(r'(\d+)/(\d+)/(\d+)', text)

# 如果你打算做大量的匹配和搜索操作的话，最好先编译正则表达式，然后再重复使用它。

[('11', '27', '2012'), ('3', '13', '2013')]

## 2.5 字符串搜索和替换

problem: 你想在字符串中搜索和匹配指定的文本模式

ans: str.replace(), re.sub()

In [2]:
# 对于简单的字面模式，直接使用 str.replace() 方法即可
text = 'yeah, but no, but yeah, but no, but yeah'
print(text.replace('year', 'yep'))

# 对于复杂的模式，请使用 re 模块中的 sub() 函数
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'
import re
# 将形式为 11/27/2012 的日期字符串改成 2012-11-27
re.sub(r'(\d+)/(\d+)/(\d+)', r'\3-\1-\2', text)
# sub() 函数中的第一个参数是被匹配的模式，第二个参数是替换模式
# 反斜杠数字比如 \3 指向前面模式的捕获组号。

yeah, but no, but yeah, but no, but yeah


'Today is 2012-11-27. PyCon starts 2013-3-13.'

In [5]:
# 如果你打算用相同的模式做多次替换，考虑先编译它来提升性能
import re
datepet = re.compile(r'(\d+)/(\d+)/(\d+)')
print(datepet.sub(r'\3-\1-\2', text))

# 对于更加复杂的替换，可以传递一个替换回调函数来代替
# 一个替换回调函数的参数是一个 match 对象，也就是 match() 或者 find() 返回的对象。 
from calendar import month_abbr
def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))
print(datepet.sub(change_date, text))

# 如果除了替换后的结果外，你还想知道有多少替换发生了，可以使用 re.subn() 来代替
newtext, n = datepet.subn(r'\3-\1-\2', text)
print(newtext, n)

Today is 2012-11-27. PyCon starts 2013-3-13.
Today is 27 Nov 2012. PyCon starts 13 Mar 2013.
Today is 2012-11-27. PyCon starts 2013-3-13. 2


## 2.6 字符串忽略大小写的搜索替换

problem: 你需要以忽略大小写的方式搜索与替换文本字符串

ans: `re.IGNORECASE` 标志参数

In [12]:
import re
text = 'UPPER PYTHON, lower python, Mixed Python'
m = re.findall('python', text, flags=re.IGNORECASE)
print(m)
m = re.sub('python', 'snake', text, flags=re.IGNORECASE)
print(m)

# 替换字符串自动跟被匹配字符串的大小写保持一致
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

m = re.sub('python', matchcase('snake'), text, flags=re.IGNORECASE)
print(m)

['PYTHON', 'python', 'Python']
UPPER snake, lower snake, Mixed snake
UPPER SNAKE, lower snake, Mixed Snake


## 2.7 最短匹配模式

problem: 你正在试着用正则表达式匹配某个文本模式，但是它找到的是模式的最长可能匹配。 而你想修改它变成查找最短的可能匹配。

ans: `?` 修饰符

In [14]:
# 这个问题一般出现在需要匹配一对分隔符之间的文本的时候(比如引号包含的字符串)
str_pat = re.compile(r'"(.*)"')
text1 = 'Computer says "no".'
m = str_pat.findall(text1)
print(m)

# 在这个例子中，模式 r'\"(.*)\"' 的意图是匹配被双引号包含的文本。 
# 但是在正则表达式中*操作符是贪婪的，因此匹配操作会查找最长的可能匹配。
text2 = 'Computer says "no." Phone says "yes."'
m = str_pat.findall(text2)
print(m)

# 为了修正这个问题，可以在模式中的*操作符后面加上?修饰符
str_pat = re.compile(r'"(.*?)"')
m = str_pat.findall(text2)
print(m)

['no']
['no." Phone says "yes.']
['no.', 'yes.']


## 2.8 多行匹配模式

problem: 你正在试着使用正则表达式去匹配一大块的文本，而你需要跨越多行去匹配。

In [21]:
import re
comment = re.compile(r'/\*(.*?)\*/')
text1 = '/* this is a comment */'
text2 = '''/* this is a
 multiline comment */
'''
m = comment.findall(text1)
print(m)
comment.findall(text2)
print(m)

[' this is a comment ']
[' this is a comment ']


In [22]:
# 为了修正这个问题，你可以修改模式字符串，增加对换行的支持
comment = re.compile(r'/\*((?:.|\n)*?)\*/')
m = comment.findall(text2)
print(m)

# (?:.|\n) 指定了一个非捕获组 (也就是它定义了一个仅仅用来做匹配，而不能通过单独捕获或者编号的组)

[' this is a\n multiline comment ']


In [23]:
# re.compile() 函数接受一个标志参数叫 re.DOTALL, 
# 它可以让正则表达式中的点(.)匹配包括换行符在内的任意字符
comment = re.compile(r'/\*(.*?)\*/', re.DOTALL)
m = comment.findall(text2)
print(m)

[' this is a\n multiline comment ']


## 2.9 将Unicode文本标准化

problem: 你正在处理Unicode字符串，需要确保所有字符串在底层有相同的表示。

ans: import unicodedata

In [5]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'
print(s1)
print(s2)
print(len(s1), len(s2))

Spicy Jalapeño
Spicy Jalapeño
14 15


In [13]:
import unicodedata

# 'NFC'表示字符应该是整体组成(比如可能的话就使用单一编码)，而'NFD'表示字符应该分解为多个组合字符表示。
t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)
print(t1 == t2)
print(ascii(t1))

t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)
print(t3 == t4)
print(ascii(t3))

True
'Spicy Jalape\xf1o'
True
'Spicy Jalapen\u0303o'


In [15]:
# 测试字符类的工具函数
t1 = unicodedata.normalize('NFD', s1)
''.join(c for c in t1 if not unicodedata.combining(c))

'Spicy Jalapeno'

## 2.10 在正则式中使用Unicode

problem: 你正在使用正则表达式处理文本，但是关注的是Unicode字符处理。

ans: 转义序列

In [21]:
# 默认情况下 re 模块已经对一些Unicode字符类有了基本的支持。 
# 比如， \\d 已经匹配任意的unicode数字字符了。
import re
num = re.compile('\d+')
# ASCII digits
m = num.match('123')
print(m)
# Arcbic digits
m = num.match('\u0661\u0662\u0663')
print(m)

<re.Match object; span=(0, 3), match='123'>
<re.Match object; span=(0, 3), match='١٢٣'>


In [ ]:
# 如果你想在模式中包含指定的Unicode字符，你可以使用Unicode字符对应的转义序列(比如 \uFFF 或者 \UFFFFFFF)
arabic = re.compile('[\u0600-\u06ff\u0750-\u077f\u08a0-\u08ff]+)

In [22]:
# 当执行匹配和搜索操作的时候，最好是先标准化并且清理所有文本为标准化格式(参考2.9小节)。 
# 但是同样也应该注意一些特殊情况，比如在忽略大小写匹配和大小写转换时的行为。
pat = re.compile('stra\u00dfe', re.IGNORECASE)
s = 'straße'
m = pat.match(s) # Matches
print(m)
m = pat.match(s.upper()) # Don't match
print(m)
print(s.upper())

<re.Match object; span=(0, 6), match='straße'>
None
STRASSE


混合使用Unicode和正则表达式通常会让你抓狂。 如果你真的打算这样做的话，最好考虑下安装第三方正则式库， 它们会为Unicode的大小写转换和其他大量有趣特性提供全面的支持，包括模糊匹配。

## 2.11 删除字符串中不需要的字符

problem: 你想去掉文本字符串开头，结尾或者中间不想要的字符，比如空白。

ans: strip(), istrip(), rstrip()

In [2]:
# Whitespace stripping
s = ' hello world \n'
print(s.strip())
print(s.lstrip())
print(s.rstrip())

# Character stripping
t = '-----hello====='
print(t.strip('-'))
print(t.strip('='))

hello world
hello world 

 hello world
hello=====
-----hello


In [5]:
# 需要注意的是去除操作不会对字符串的中间的文本产生任何影响
s = ' hello     world \n'
s = s.strip()
print(s)

# 如果你想处理中间的空格，那么你需要求助其他技术:replace, re
print(s.replace(' ', ''))
import re
print(re.sub('\s+', ' ', s))

hello     world
helloworld
hello world


In [ ]:
# 通常情况下你想将字符串 strip 操作和其他迭代操作相结合，比如从文件中读取多行数据。 
# 如果是这样的话，那么生成器表达式就可以大显身手了。
with open(filename) as f:
    lines = (line.strip() for line in f)
    for line in lines:
        print(line)

## 2.12 审查清理文本字符串

problem: 一些无聊的幼稚黑客在你的网站页面表单中输入文本”pýtĥöñ”，然后你想将这些字符清理掉。

ans: str.translate()

In [14]:
s = 'pýtĥöñ\fis\tawesome\r\n'
# 第一步是清理空白字符, 先创建一个小的转换表格然后使用 translate() 方法
remap = {
    ord('\t') : ' ',
    ord('\f') : ' ',
    ord('\r') : None  # Deleted
}
a = s.translate(remap)
print(a)

# 删除所有的和音符
import unicodedata
import sys
# 创建字典，每个Unicode和音符作为键，对应的值全部为 None
cmb_chrs = dict.fromkeys(c for c in range(sys.maxunicode)
                        if unicodedata.combining(chr(c)))
# print(cmb_chrs)
b = unicodedata.normalize('NFD', a)
print(b)
c = b.translate(cmb_chrs)
print(c)

pýtĥöñ is awesome

pýtĥöñ is awesome

python is awesome



In [20]:
# 构造一个将所有Unicode数字字符映射到对应的ASCII字符上的表格
digitmap = {c : ord('0') + unicodedata.digit(chr(c))
    for c in range(sys.maxunicode)
    if unicodedata.category(chr(c)) == 'Nd'}
print(len(digitmap))
# print(digitmap)
# Arabic digits
x = '\u0661\u0662\u0663'
x.translate(digitmap)

630


'123'

In [22]:
# I/O解码与编码函数:encode(), decode()
a = 'pýtĥöñ is awesome\n'
b = unicodedata.normalize('NFD', a)
b.encode('ascii', 'ignore').decode('ascii')
# 这里的标准化操作将原来的文本分解为单独的和音符。接下来的ASCII编码/解码只是简单的一下子丢弃掉那些字符

'python is awesome\n'

文本字符清理一个最主要的问题应该是运行的性能。一般来讲，代码越简单运行越快。 对于简单的替换操作， str.replace() 方法通常是最快的。

## 2.13 字符串对齐

problem: 你想通过某种对齐方式来格式化字符串

ans: ljust(), rjust(), center(); format()

In [34]:
text = 'Hello World'
s = text.ljust(20)
print(s)
s = text.rjust(20)
print(s)
s = text.center(20)
print(s)

# 所有这些方法都能接受一个可选的填充字符
s = text.rjust(20, '=')
print(s)
s = text.center(20, '*')
print(s)

Hello World         
         Hello World
    Hello World     
=========Hello World
****Hello World*****


In [46]:
# 函数 format() 同样可以用来很容易的对齐字符串。 你要做的就是使用 <, > 或者 ^ 字符后面紧跟一个指定的宽度
format(text, '>20')
format(text, '<20')
format(text, '^20')

format(text, '=>20')
format(text, '*^20')

# 当格式化多个值的时候，这些格式代码也可以被用在 format() 方法中
print('{:>10s} {:>10s}'.format('Hello', 'World'))

# format() 函数的一个好处是它不仅适用于字符串。它可以用来格式化任何值
x = 1.2345
format(x, '>10')
format(x, '^10.2f')

Hello      World


'   1.23   '

## 2.14 合并拼接字符串

problem: 你想将几个小的字符串合并为一个大的字符串

ans: join(), + 

In [5]:
parts = ['Is', 'Chicago', 'Not', 'Chicago?']
print(' '.join(parts))
print(','.join(parts))
print(''.join(parts))

# 如果你仅仅只是合并少数几个字符串，使用加号(+)通常已经足够了
a = 'Is Chicago'
b = 'Not Chicago?'
a += ' ' + b
print(a)
print('{} {}'.format(a, b))
print(a + ' ' + b)

# 如果你想在源码中将两个字面字符串合并起来，你只需要简单的将它们放到一起，不需要用加号(+)
a = 'Hello' 'World'
print(a)

Is Chicago Not Chicago?
Is,Chicago,Not,Chicago?
IsChicagoNotChicago?
Is Chicago Not Chicago?
Is Chicago Not Chicago? Not Chicago?
Is Chicago Not Chicago? Not Chicago?
HelloWorld


In [ ]:
# 这种写法会比使用 join() 方法运行的要慢一些，因为每一次执行+=操作的时候会创建一个新的字符串对象。 
# 你最好是先收集所有的字符串片段然后再将它们连接起来。
s = ''
for p in parts: 
    s += p

In [8]:
# 一个相对比较聪明的技巧是利用生成器表达式(参考1.19小节)转换数据为字符串的同时合并字符串
data = ['ACME', 50, 91.1]
print(','.join(str(d) for d in data))

a = 'a'
b = 'b'
c = 'c'

# 同样还得注意不必要的字符串连接操作
print(a + ';' + b + ';' + c) # ugly
print(';'.join([a, b, c])) # still ugly
print(a, b, c, sep=';') # better

ACME,50,91.1
a;b;c
a;b;c
a;b;c


In [ ]:
# 当混合使用I/O操作和字符串连接操作的时候，有时候需要仔细研究你的程序
# Version 1 (string concatenation)
f.write(chunk1 + chunk2)

# Version 2 (separate I/O operations)
f.write(chunk1)
f.write(chunk2)

如果两个字符串很小，那么第一个版本性能会更好些，因为I/O系统调用天生就慢。 另外一方面，如果两个字符串很大，那么第二个版本可能会更加高效， 因为它避免了创建一个很大的临时结果并且要复制大量的内存块数据。

如果你准备编写构建大量小字符串的输出代码， 你最好考虑下使用生成器函数，利用yield语句产生输出片段。

In [9]:
def sample():
    yield 'Is'
    yield 'Chicago'
    yield 'Not'
    yield 'Chicago?'

## 2.15 字符串中插入变量

problem: 你想创建一个内嵌变量的字符串，变量被它的值所表示的字符串替换掉。

ans: 

In [14]:
# Python并没有对在字符串中简单替换变量值提供直接的支持。 但是通过使用字符串的 format() 方法来解决这个问题。
s = '{name} has {n} messages.'
print(s.format(name='Guido', n=37))

Guido has 37 messages.


In [16]:
# 或者，如果要被替换的变量能在变量域中找到， 那么你可以结合使用 format_map() 和 vars()
name = 'Guido'
n = 37
print(s.format_map(vars()))

# vars() 还有一个有意思的特性就是它也适用于对象实例。
class Info:
    def __init__(self, name, n):
        self.name = name
        self.n = n

a = Info('Guido', 37)
print(s.format_map(vars()))

Guido has 37 messages.
Guido has 37 messages.


In [17]:
# format 和 format_map() 的一个缺陷就是它们并不能很好的处理变量缺失的情况，比如
s.format(name='Guido')

KeyError: 'n'

In [21]:
# 一种避免这种错误的方法是另外定义一个含有 __missing__() 方法的字典对象
class safesub(dict):
    """防止key找不到"""
    def __missing__(self, key):
        return '{' + key + '}'

# 现在你可以利用这个类包装输入后传递给 format_map() 
# del n   # Make sure n is undefined
print(s.format_map(safesub(vars())))

# 如果你发现自己在代码中频繁的执行这些步骤，你可以将变量替换步骤用一个工具函数封装起来。就像下面这样：
import sys
def sub(text):
    # sub() 函数使用 sys._getframe(1) 返回调用者的栈帧, 可以从中访问属性 f_locals 来获得局部变量。 
    # f_locals 是一个复制调用函数的本地变量的字典
    return text.format_map(safesub(sys._getframe(1).f_locals))

# 现在你可以像下面这样写了：
name = 'Guido'
n = 37
print(sub('Hello {name}'))
print(sub('You have {n} messages.'))
print(sub('Your favorite color is {color}.'))

Guido has {n} messages.
Hello Guido
You have 37 messages.
Your favorite color is {color}.


## 2.16 以指定列宽格式化字符串

problem: 你有一些长字符串，想以指定的列宽将它们重新格式化。

ans: `textwrap` 模块

In [30]:
s = "Look into my eyes, look into my eyes, the eyes, the eyes, \
the eyes, not around the eyes, don't look around the eyes, \
look into my eyes, you're under."

import textwrap

print(textwrap.fill(s, 70))
print(textwrap.fill(s, 40))

print(textwrap.fill(s, 40, initial_indent='     '))
print(textwrap.fill(s, 40, subsequent_indent='      '))
# fill() 方法接受一些其他可选参数来控制tab，语句结尾等。

Look into my eyes, look into my eyes, the eyes, the eyes, the eyes,
not around the eyes, don't look around the eyes, look into my eyes,
you're under.
Look into my eyes, look into my eyes,
the eyes, the eyes, the eyes, not around
the eyes, don't look around the eyes,
look into my eyes, you're under.
     Look into my eyes, look into my
eyes, the eyes, the eyes, the eyes, not
around the eyes, don't look around the
eyes, look into my eyes, you're under.
Look into my eyes, look into my eyes,
      the eyes, the eyes, the eyes, not
      around the eyes, don't look around
      the eyes, look into my eyes,
      you're under.


`textwrap` 模块对于字符串打印是非常有用的，特别是当你希望输出自动匹配终端大小的时候。 你可以使用 `os.get_terminal_size()` 方法来获取终端的大小尺寸。

In [32]:
import os
os.get_terminal_size().columns

## 2.17 在字符串中处理html和xml

problem: 你想将HTML或者XML实体如 &entity; 或 &#code; 替换为对应的文本。 再者，你需要转换文本中特定的字符(比如<, >, 或 &)。

ans: `html.escape()`, `HTMLParser.unescape()`

In [2]:
# 如果你想替换文本字符串中的 ‘<’ 或者 ‘>’ ，使用 html.escape() 函数
s = 'Elements are written as "<tag>text</tag>".'
import html 
print(s)
print(html.escape(s))

# Disable escaping of quote
print(html.escape(s, quote=False))

Elements are written as "<tag>text</tag>".
Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.
Elements are written as "&lt;tag&gt;text&lt;/tag&gt;".


In [4]:
# 如果你正在处理的是ASCII文本，并且想将非ASCII文本对应的编码实体嵌入进去， 
# 可以给某些I/O函数传递参数 errors='xmlcharrefreplace' 来达到这个目。
s = 'Spicy Jalapeño'
s.encode('ascii', errors='xmlcharrefreplace')

b'Spicy Jalape&#241;o'

In [6]:
# 如果你接收到了一些含有编码值的原始文本，需要手动去做替换， 通常你只需要使用HTML或者XML解析器的一些相关工具函数/方法即可
s = 'Spicy &quot;Jalape&#241;o&quot.'
from html.parser import HTMLParser
p = HTMLParser()
print(p.unescape(s))

t = 'The prompt is &gt;&gt;&gt;'
from xml.sax.saxutils import unescape
print(unescape(t))

Spicy "Jalapeño".
The prompt is >>>


## 2.18 字符串令牌解析

problem: 你有一个字符串，想从左至右将其解析为一个令牌流。

ans: 

## 2.19 实现一个简单的递归下降分析器

problem: 你想根据一组语法规则解析文本并执行命令，或者构造一个代表输入的抽象语法树。 如果语法非常简单，你可以不去使用一些框架，而是自己写这个解析器。

## 2.20 字节字符串上的字符串操作

problem: 你想在字节字符串上执行普通的文本操作(比如移除，搜索和替换)。

ans: 字节字符串同样也支持大部分和文本字符串一样的内置操作。

In [11]:
data = b'Hello World'
print(data[0:5])
print(data.startswith(b'Hello'))
print(data.split())
print(data.replace(b'Hello', b'Hello Cruel'))

b'Hello'
True
[b'Hello', b'World']
b'Hello Cruel World'


In [13]:
# 这些操作同样也适用于字节数组
data = bytearray(b'Hello World')
print(data[0:5])
print(data.startswith(b'Hello'))
print(data.split())
print(data.replace(b'Hello', b'Hello Cruel'))

bytearray(b'Hello')
True
[bytearray(b'Hello'), bytearray(b'World')]
bytearray(b'Hello Cruel World')


In [14]:
# 你可以使用正则表达式匹配字节字符串，但是正则表达式本身必须也是字节串。
data = b'FOO:BAR,SPAM'
import re
# re.split('[:,]', data)
re.split(b'[:,]', data)

[b'FOO', b'BAR', b'SPAM']

大多数情况下，在文本字符串上的操作均可用于字节字符串。 然而，这里也有一些需要注意的不同点。

In [17]:
# 1. 字节字符串的索引操作返回整数而不是单独字符。
a = 'Hello World' # Text string
print(a[0])
print(a[1])

b = b'Hello World' # Byte string
print(b[0])
print(b[1])

H
e
72
101


In [23]:
# 2. 字节字符串不会提供一个美观的字符串表示，也不能很好的打印出来，除非它们先被解码为一个文本字符串。
s = b'Hello World'
print(s)   # Observe b'...'
print(s.decode('ascii'))

# 类似的，也不存在任何适用于字节字符串的格式化操作
# print(b'%10s %10d %10.2f' % (b'ACME', 100, 490.1))

# 如果你想格式化字节字符串，你得先使用标准的文本字符串，然后将其编码为字节字符串。
print('{:10s} {:10d} {:10.2f}'.format('ACME', 100, 490.1).encode('ascii'))

b'Hello World'
Hello World
b'ACME              100     490.10'


In [27]:
# 3. 使用字节字符串可能会改变一些操作的语义，特别是那些跟文件系统有关的操作。
# Write a UTF-8 filename
with open('jalape\xf1o.txt', 'w') as f:
    f.write('spicy')
# Get a directory listing
import os
os.listdir('.')   # Text string (names are decoded)
os.listdir(b'.') # Byte string (names left as bytes)

[b'.git',
 b'.gitignore',
 b'1.data-strcture-and-algorithm.ipynb',
 b'2.string-and-text.ipynb',
 b'jalape\xc3\xb1o.txt',
 b'README.md']